#### Applying Genetic Algorithm on ANN which is trained on Iris dataset

In [ ]:
#iris - ann
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a neural network
model = Sequential([
    Dense(8, input_dim=4, activation='relu'),
    Dense(4, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define genetic algorithm parameters
population_size = 20  # Increased population size
generations = 20  # Increased number of generations
mutation_rate = 0.05  # Adjusted mutation rate

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to evaluate fitness (loss)
def evaluate_fitness(individual):
    model.set_weights(individual)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
    return loss

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))
print(f"Test set accuracy: {test_accuracy:.2f}")

# Print the final weights of the best individual
for i, weight_matrix in enumerate(best_weights):
    print(f"Layer {i+1} weights:\n{weight_matrix}\n")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Early stopping at generation 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Test set accuracy: 0.63
Layer 1 weights:
[[ 0.18058989  0.74589167  0.00303259  0.6131223   0.31756673 -0.12774694
   0.6983913   0.89690661]
 [-0.02115007 -0.24883409  0.97533602  0.15656028  0.78910445  0.41115035
  -0.14957299 -0.63884931]
 [ 0.13589046  0.08326843 -0.93210804 -0.48422463 -0.27961872  0.66986047
   0.94211649  0.88853298]
 [-0.05157157 -0.45538535 -0.96321865  0.82859761  0.65783095 -0.92598473
   0.19253976 -0.53998233]]

Layer 2 weights:
[-0.75886623 -0.8460936   0.39257755 -0.32025007  0.44953354 -0.86928732
 -0.36941932  0.07898258]

Layer 3 weights:
[[ 0.92529683 -0.36249499  0.25178275  0.7719555 ]
 [ 0.23172638 -0.53408105 -0.95119844  0.09845333]
 [-0.95746118  0.74940335 -0.83440266  0.8781354 ]
 [ 0.59756647 -0.22140877 -0.42261253  0.53437658]
 [-0.19613817 -0.04024876  0.25501093 -0.97121302]
 [ 0.96816694  0.53654683 -0.9185424  -0.57011924]
 [ 0.4751646  -0.47547191 -0.77905177 -0.897

#### Applying Genetic Algorithm on CNN which is trained on IMDB dataset

In [ ]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the IMDB dataset
max_features = 10000  # Number of words to consider as features
maxlen = 200  # Cut reviews after 200 words

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Further split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create an ANN model for IMDB
model = Sequential([
    Embedding(max_features, 32, input_length=maxlen),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize model weights by running a single forward pass
model.predict(X_train[:1])  # Pass a single sample to initialize the model's weights

# Genetic Algorithm parameters
population_size = 20  # Number of individuals in population
generations = 20  # Number of generations
mutation_rate = 0.05  # Mutation rate

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to evaluate fitness (loss)
def evaluate_fitness(individual):
    model.set_weights(individual)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
    return loss

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_accuracy = np.mean((test_predictions > 0.5).astype('int') == y_test.reshape(-1, 1))
print(f"Test set accuracy: {test_accuracy:.2f}")

# Print the final weights of the best individual
for i, weight_matrix in enumerate(best_weights):
    print(f"Layer {i+1} weights:\n{weight_matrix}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Test set accuracy: 0.56
Layer 1 weights:
[[ 0.71281658  0.18097507  0.2522776  ...  0.45739216 -0.35118316
   0.42563227]
 [ 0.26282334  0.03703197  0.96925217 ...  0.35118023 -0.06573472
   0.20589389]
 [-0.94525599 -0.42875561  0.6972702  ... -0.74727591 -0.7206741
   0.99939235]
 ...
 [-0.69930307  0.32976026 -0.68157142 ... -0.37805061 -0.65816606
   0.65012886]
 [-0.6011073   0.38252289 -0.4586843  ...  0.14753509 -0.09051445
   0.50697645]
 [-0.42316076  0.48934615  0.12527095 ...  0.20445338 -0.39238096
  -0.34842802]]

Layer 2 weights:
[[ 0.66740539 -0.7831877   0.78245429  0.28469872 -1.52049908  1.41778455
  -0.09413383  0.10187639 -0.40188577  0.77950599  0.82991201  0.7563814
   1.19717939  0.97378531 -0.3419992  -0.38669786]
 [-0.65905467  0.85690143 -0.39196833  0.27590061  0.56434862 -0.34818243
   0.88772949  0.29988508 -0.01952688  0.79581645  0.57616341 -0.82426832
  -0.62778351  0.5056140

In [ ]:
# import numpy as np
# import torch
# from torch.utils.data import DataLoader, Dataset
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from datasets import load_dataset

# # Load and preprocess the IMDB dataset
# dataset = load_dataset('imdb')

# # Preprocessing and feature extraction
# train_texts = dataset['train']['text']
# train_labels = dataset['train']['label']
# test_texts = dataset['test']['text']
# test_labels = dataset['test']['label']

# # Split train into train and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# # Vectorize the text data
# vectorizer = CountVectorizer(stop_words='english', max_features=1000)
# X_train = vectorizer.fit_transform(train_texts).toarray()
# X_val = vectorizer.transform(val_texts).toarray()
# X_test = vectorizer.transform(test_texts).toarray()

# # Convert to PyTorch tensors
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
# X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
# y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
# y_val_tensor = torch.tensor(val_labels, dtype=torch.long)
# y_test_tensor = torch.tensor(test_labels, dtype=torch.long)

# class IMDBDataset(Dataset):
#     def __init__(self, texts, labels):
#         self.texts = texts
#         self.labels = labels

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         return {
#             'inputs': self.texts[idx],
#             'labels': self.labels[idx]
#         }

# # Prepare datasets and dataloaders
# train_dataset = IMDBDataset(X_train_tensor, y_train_tensor)
# val_dataset = IMDBDataset(X_val_tensor, y_val_tensor)
# test_dataset = IMDBDataset(X_test_tensor, y_test_tensor)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32)
# test_loader = DataLoader(test_dataset, batch_size=32)

# # Define the ANN model
# class ANNModel(torch.nn.Module):
#     def __init__(self, input_dim, hidden_units, output_dim, weights=None):
#         super(ANNModel, self).__init__()
#         self.fc1 = torch.nn.Linear(input_dim, hidden_units)
#         self.fc2 = torch.nn.Linear(hidden_units, hidden_units)
#         self.fc3 = torch.nn.Linear(hidden_units, output_dim)
#         self.relu = torch.nn.ReLU()

#         # Assign the custom weights to the first layer if provided
#         if weights is not None:
#             with torch.no_grad():
#                 self.fc1.weight.copy_(weights)

#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# def create_model(hidden_units, learning_rate, weights=None):
#     input_dim = X_train.shape[1]  # Number of features
#     output_dim = 2  # Binary classification
#     model = ANNModel(input_dim, int(hidden_units), output_dim, weights)
#     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#     return model, optimizer

# # Objective Function: Minimize Validation Loss
# def objective_function(hyperparams):
#     hidden_units = int(hyperparams[0])
#     learning_rate = hyperparams[1]

#     # Initialize weights with correct dimensions: (hidden_units, input_dim)
#     weights = torch.randn((hidden_units, X_train.shape[1]), dtype=torch.float32)

#     model, optimizer = create_model(hidden_units, learning_rate, weights)
#     criterion = torch.nn.CrossEntropyLoss()

#     # Train the model for one epoch
#     model.train()
#     for batch in train_loader:
#         inputs = batch['inputs']
#         labels = batch['labels']
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#     # Evaluate on validation set
#     model.eval()
#     preds = []
#     labels_list = []
#     with torch.no_grad():
#         for batch in val_loader:
#             inputs = batch['inputs']
#             labels = batch['labels']
#             outputs = model(inputs)
#             preds += torch.argmax(outputs, dim=1).tolist()
#             labels_list += labels.tolist()

#     accuracy = accuracy_score(labels_list, preds)
#     return 1 - accuracy  # Minimize this value

# # Bat Algorithm Implementation
# class BatAlgorithm:
#     def __init__(self, num_bats, max_iter, bounds, alpha=0.9, gamma=0.9):
#         self.num_bats = num_bats
#         self.max_iter = max_iter
#         self.bounds = bounds
#         self.alpha = alpha
#         self.gamma = gamma

#         # Initialize bats
#         self.bats = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_bats, bounds.shape[0]))
#         self.velocities = np.zeros((num_bats, bounds.shape[0]))
#         self.fitness = np.zeros(num_bats)
#         self.frequencies = np.zeros(num_bats)

#         # Best bat initialization
#         self.best_bat = self.bats[0]
#         self.best_fitness = float("inf")

#     def optimize(self, objective_function):
#         for iter in range(self.max_iter):
#             for i in range(self.num_bats):
#                 # Frequency update
#                 self.frequencies[i] = np.random.uniform(0, 1)
#                 # Velocity update
#                 self.velocities[i] = self.velocities[i] + (self.bats[i] - self.best_bat) * self.frequencies[i]
#                 # Position update
#                 candidate_bat = self.bats[i] + self.velocities[i]
#                 candidate_bat = np.clip(candidate_bat, self.bounds[:, 0], self.bounds[:, 1])

#                 # Evaluate candidate
#                 fitness = objective_function(candidate_bat)

#                 # Update if new solution is better
#                 if fitness < self.fitness[i]:
#                     self.bats[i] = candidate_bat
#                     self.fitness[i] = fitness

#                 # Update the best bat
#                 if self.fitness[i] < self.best_fitness:
#                     self.best_bat = self.bats[i]
#                     self.best_fitness = self.fitness[i]

#             print(f"Iteration {iter+1}/{self.max_iter}, Best Fitness: {self.best_fitness}")

#         return self.best_bat

# # Hyperparameter bounds: hidden units (4 to 64), learning rate (1e-6 to 1e-2)
# bounds = np.array([[4, 64], [1e-6, 1e-2]])

# # Initialize and run Bat Algorithm
# bat_algorithm = BatAlgorithm(num_bats=10, max_iter=10, bounds=bounds, alpha=0.9, gamma=0.9)
# best_hyperparams = bat_algorithm.optimize(objective_function)
# print("Best hyperparameters found:", best_hyperparams)

# # Extract the best hyperparameters
# best_hidden_units = int(best_hyperparams[0])
# best_lr = best_hyperparams[1]

# # Initialize final model with the best hyperparameters
# # Initialize weights with correct dimensions: (best_hidden_units, X_train.shape[1])
# final_weights = torch.randn((best_hidden_units, X_train.shape[1]), dtype=torch.float32)
# final_model, final_optimizer = create_model(best_hidden_units, best_lr, final_weights)

# # Train the final model
# num_epochs = 3  # You can adjust this based on your needs
# criterion = torch.nn.CrossEntropyLoss()

# for epoch in range(num_epochs):
#     final_model.train()
#     for batch in train_loader:
#         inputs = batch['inputs']
#         labels = batch['labels']
#         final_optimizer.zero_grad()
#         outputs = final_model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         final_optimizer.step()
#     print(f"Epoch {epoch+1} completed.")

# # Evaluate the final model on the test set
# final_model.eval()
# test_loader = DataLoader(test_dataset, batch_size=32)
# preds = []
# labels_list = []

# with torch.no_grad():
#     for batch in test_loader:
#         inputs = batch['inputs']
#         labels = batch['labels']
#         outputs = final_model(inputs)
#         preds += torch.argmax(outputs, dim=1).tolist()
#         labels_list += labels.tolist()

# # Calculate and print accuracy
# accuracy = accuracy_score(labels_list, preds)
# print(f"Final model accuracy: {accuracy:.2f}")


Iteration 1/10, Best Fitness: 0.0
Iteration 2/10, Best Fitness: 0.0
Iteration 3/10, Best Fitness: 0.0
Iteration 4/10, Best Fitness: 0.0
Iteration 5/10, Best Fitness: 0.0
Iteration 6/10, Best Fitness: 0.0
Iteration 7/10, Best Fitness: 0.0
Iteration 8/10, Best Fitness: 0.0
Iteration 9/10, Best Fitness: 0.0
Iteration 10/10, Best Fitness: 0.0
Best hyperparameters found: [5.16831170e+01 3.94161812e-03]
Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Final model accuracy: 0.84


#### Applying Genetic Algorithm on CNN with dropout layer which is trained on IMDB dataset

In [ ]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the IMDB dataset
max_features = 10000  # Number of words to consider as features
maxlen = 200  # Cut reviews after 200 words

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Further split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create an enhanced ANN model for IMDB
model = Sequential([
    Embedding(max_features, 32, input_length=maxlen),
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=4),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize model weights by running a single forward pass
model.predict(X_train[:1])  # Pass a single sample to initialize the model's weights

# Genetic Algorithm parameters
population_size = 20  # Number of individuals in population
generations = 20  # Number of generations
mutation_rate = 0.05  # Mutation rate

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to evaluate fitness (loss)
def evaluate_fitness(individual):
    model.set_weights(individual)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
    return loss

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_accuracy = np.mean((test_predictions > 0.5).astype('int') == y_test.reshape(-1, 1))
print(f"Test set accuracy: {test_accuracy:.2f}")

# Print the final weights of the best individual
for i, weight_matrix in enumerate(best_weights):
    print(f"Layer {i+1} weights:\n{weight_matrix}\n")


1/1 [==============================] - 0s 156ms/step
Early stopping at generation 11
782/782 [==============================] - 2s 2ms/step
Test set accuracy: 0.51
Layer 1 weights:
[[-0.35988579  0.84897452 -0.40005202 ... -0.2844398  -0.14436807
   0.4301523 ]
 [-0.6232936   0.46431354 -0.42763575 ...  0.39351808  0.4156008
  -0.46199049]
 [-0.63222245 -0.66246607 -0.10488483 ... -0.5961557  -0.44536021
   0.54922633]
 ...
 [ 0.63144705  0.43628677 -0.35724277 ... -0.29339546 -0.20199538
  -0.62600855]
 [ 0.32735023 -0.10112649  0.99901866 ... -0.63706175 -0.7228877
  -0.52927442]
 [-0.73613596 -0.84538998  0.76163167 ... -0.51312007 -0.06823992
  -0.72200804]]

Layer 2 weights:
[[[-0.60753341  0.3178871  -1.21406276 ...  0.58428471  1.0777442
    0.92880067]
  [ 0.25166232 -0.01317954 -0.3102024  ... -0.4782268   0.81053919
   -0.3312514 ]
  [-0.15808124 -0.93199049  0.52287361 ... -0.64348712  0.52927495
    0.60249956]
  ...
  [-0.62032117 -0.59000462  0.88400106 ... -0.54777161  0

In [ ]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the IMDB dataset
max_features = 10000  # Number of words to consider as features
maxlen = 200  # Cut reviews after 200 words

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Further split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create an enhanced ANN model for IMDB
model = Sequential([
    Embedding(max_features, 32, input_length=maxlen),
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=4),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initialize model weights by running a single forward pass
model.predict(X_train[:1])  # Pass a single sample to initialize the model's weights

# Genetic Algorithm parameters
population_size = 20  # Number of individuals in population
generations = 20  # Number of generations
mutation_rate = 0.05  # Mutation rate

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to evaluate fitness (loss)
def evaluate_fitness(individual):
    model.set_weights(individual)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
    return loss

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_accuracy = np.mean((test_predictions > 0.5).astype('int') == y_test.reshape(-1, 1))
print(f"Test set accuracy: {test_accuracy:.2f}")

# Print the final weights of the best individual
for i, weight_matrix in enumerate(best_weights):
    print(f"Layer {i+1} weights:\n{weight_matrix}\n")


1/1 [==============================] - 0s 144ms/step
Early stopping at generation 11
782/782 [==============================] - 2s 2ms/step
Test set accuracy: 0.51
Layer 1 weights:
[[-0.35988579  0.84897452 -0.40005202 ... -0.2844398  -0.14436807
   0.4301523 ]
 [-0.6232936   0.46431354 -0.42763575 ...  0.39351808  0.4156008
  -0.46199049]
 [-0.63222245 -0.66246607 -0.10488483 ... -0.5961557  -0.44536021
   0.54922633]
 ...
 [ 0.63144705  0.43628677 -0.35724277 ... -0.29339546 -0.20199538
  -0.62600855]
 [ 0.32735023 -0.10112649  0.99901866 ... -0.63706175 -0.7228877
  -0.52927442]
 [-0.73613596 -0.84538998  0.76163167 ... -0.51312007 -0.06823992
  -0.72200804]]

Layer 2 weights:
[[[-0.60753341  0.3178871  -1.21406276 ...  0.58428471  1.0777442
    0.92880067]
  [ 0.25166232 -0.01317954 -0.3102024  ... -0.4782268   0.81053919
   -0.3312514 ]
  [-0.15808124 -0.93199049  0.52287361 ... -0.64348712  0.52927495
    0.60249956]
  ...
  [-0.62032117 -0.59000462  0.88400106 ... -0.54777161  0

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 786.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.8/446.8 kB 23.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0


#### Applying Genetic Algorithm with fitness function of MSE on ANN which is trained on Iris dataset

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a neural network
model = Sequential([
    Dense(8, input_dim=4, activation='relu'),
    Dense(4, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model using mean squared error (MSE)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to evaluate fitness using MSE
def evaluate_fitness(individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)
    mse = np.mean(np.square(predictions - y_train))  # MSE for training set predictions
    return mse

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set using MSE
test_predictions = model.predict(X_test)
test_mse = np.mean(np.square(test_predictions - y_test))  # MSE for test set
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Early stopping at generation 15
1/1 [==============================] - 0s 23ms/step
Test set MSE: 0.0380
Test set accuracy: 0.93


#### Applying Genetic Algorithm with hybrid fitness function of MSE and CCE on ANN which is trained on Iris dataset

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.losses import categorical_crossentropy

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a neural network
model = Sequential([
    Dense(8, input_dim=4, activation='relu'),
    Dense(4, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05
mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to calculate MSE for fitness
def calculate_mse(predictions, true_values):
    return np.mean(np.square(predictions - true_values))

# Function to calculate CCE for fitness
def calculate_cce(predictions, true_values):
    return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# Hybrid fitness function combining MSE and CCE
def evaluate_fitness(individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)

    # Calculate MSE
    mse = calculate_mse(predictions, y_train)

    # Calculate CCE
    cce = calculate_cce(predictions, y_train)

    # Combine MSE and CCE with weights
    fitness = mse_weight * mse + cce_weight * cce
    return fitness

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

# Set the best weights found
best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.losses import categorical_crossentropy

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a neural network
model = Sequential([
    Dense(8, input_dim=4, activation='relu'),
    Dense(4, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05
mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to calculate MSE for fitness
def calculate_mse(predictions, true_values):
    return np.mean(np.square(predictions - true_values))

# Function to calculate CCE for fitness
def calculate_cce(predictions, true_values):
    return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# Hybrid fitness function combining MSE and CCE
def evaluate_fitness(individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)

    # Calculate MSE
    mse = calculate_mse(predictions, y_train)

    # Calculate CCE
    cce = calculate_cce(predictions, y_train)

    # Combine MSE and CCE with weights
    fitness = mse_weight * mse + cce_weight * cce
    return fitness

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

# Set the best weights found
best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_mse = calculate_mse(test_predictions, y_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_mse = calculate_mse(test_predictions, y_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


1/1 [==============================] - 0s 24ms/step
Test set MSE: 0.0486
Test set accuracy: 0.97


#### Applying Genetic Algorithm with hybrid fitness function of MSE and CCE on ANN with more layers which is trained on Iris dataset

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.losses import categorical_crossentropy

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a larger neural network architecture
model = Sequential([
    Dense(64, input_dim=4, activation='relu'),   # Increased units
    Dense(128, activation='relu'),                # Increased units
    Dense(64, activation='relu'),                 # Increased units
    Dense(32, activation='relu'),                 # New layer added
    Dense(3, activation='softmax')                # Output layer for 3 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05
mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to calculate MSE for fitness
def calculate_mse(predictions, true_values):
    return np.mean(np.square(predictions - true_values))

# Function to calculate CCE for fitness
def calculate_cce(predictions, true_values):
    return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# Hybrid fitness function combining MSE and CCE
def evaluate_fitness(individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)

    # Calculate MSE
    mse = calculate_mse(predictions, y_train)

    # Calculate CCE
    cce = calculate_cce(predictions, y_train)

    # Combine MSE and CCE with weights
    fitness = mse_weight * mse + cce_weight * cce
    return fitness

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

# Set the best weights found
best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_mse = calculate_mse(test_predictions, y_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Early stopping at generation 15
1/1 [==============================] - 0s 23ms/step
Test set MSE: 0.1942
Test set accuracy: 0.70


#### Applying Genetic Algorithm with hybrid fitness function of MSE and CCE on CNN which is trained on Iris dataset

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.losses import categorical_crossentropy

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Reshape X for CNN input (samples, height, width, channels)
X = X.reshape(-1, 2, 2, 1)  # Reshape to (samples, 2, 2, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a CNN model
def create_model():
    model = Sequential([
        Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(2, 2, 1)),
        MaxPooling2D(pool_size=(1, 1)),
        Conv2D(64, kernel_size=(1, 1), activation='relu'),
        MaxPooling2D(pool_size=(1, 1)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')
    ])
    return model

# Genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05
mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# Function to create random weights for the model
def create_individual(model):
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to calculate MSE for fitness
def calculate_mse(predictions, true_values):
    return np.mean(np.square(predictions - true_values))

# Function to calculate CCE for fitness
def calculate_cce(predictions, true_values):
    return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# Hybrid fitness function combining MSE and CCE
def evaluate_fitness(model, individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)

    # Calculate MSE and CCE
    mse = calculate_mse(predictions, y_train)
    cce = calculate_cce(predictions, y_train)

    # Combine MSE and CCE with weights
    fitness = mse_weight * mse + cce_weight * cce
    return fitness

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
model = create_model()
population = [create_individual(model) for _ in range(population_size)]

# Genetic algorithm loop
for generation in range(generations):
    fitness_scores = [evaluate_fitness(model, ind) for ind in population]

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

# Set the best weights found
best_weights = population[np.argmin([evaluate_fitness(model, ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_mse = calculate_mse(test_predictions, y_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


1/1 [==============================] - 0s 23ms/step
Test set MSE: 0.1693
Test set accuracy: 0.73


#### Applying Genetic Algorithm with hybrid fitness function of MSE and CCE on CNN which is trained on IMDB dataset

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.losses import categorical_crossentropy

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the IMDB dataset
max_features = 10000  # Top 10,000 words
maxlen = 200  # Maximum sequence length

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# One-hot encode the labels
encoder = OneHotEncoder(sparse=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# Create a CNN model
def create_model():
    model = Sequential([
        Embedding(max_features, 128, input_length=maxlen),
        Conv1D(64, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(128, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # Binary classification
    ])
    return model

# Genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05
mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# Function to create random weights for the model
def create_individual(model):
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to calculate MSE for fitness
def calculate_mse(predictions, true_values):
    return np.mean(np.square(predictions - true_values))

# Function to calculate CCE for fitness
def calculate_cce(predictions, true_values):
    return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# Hybrid fitness function combining MSE and CCE
def evaluate_fitness(model, individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)

    # Calculate MSE and CCE
    mse = calculate_mse(predictions, y_train)
    cce = calculate_cce(predictions, y_train)

    # Combine MSE and CCE with weights
    fitness = mse_weight * mse + cce_weight * cce
    return fitness

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
model = create_model()
population = [create_individual(model) for _ in range(population_size)]

# Genetic algorithm loop
for generation in range(generations):
    fitness_scores = [evaluate_fitness(model, ind) for ind in population]

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

# Set the best weights found
best_weights = population[np.argmin([evaluate_fitness(model, ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_mse = calculate_mse(test_predictions, y_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")


17464789/17464789 [==============================] - 0s 0us/step


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


782/782 [==============================] - 3s 4ms/step
Test set MSE: 0.4892
Test set accuracy: 0.51


#### Applying Genetic Algorithm with Hybrid Fitness Function of MSE and CCE on CNN Trained on IMDB Dataset

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.losses import categorical_crossentropy

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a larger neural network architecture
model = Sequential([
    Dense(64, input_dim=4, activation='relu'),   # Increased units
    Dense(128, activation='relu'),                # Increased units
    Dense(64, activation='relu'),                 # Increased units
    Dense(32, activation='relu'),                 # New layer added
    Dense(3, activation='softmax')                # Output layer for 3 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05
mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to calculate MSE for fitness
def calculate_mse(predictions, true_values):
    return np.mean(np.square(predictions - true_values))

# Function to calculate CCE for fitness
def calculate_cce(predictions, true_values):
    return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# Hybrid fitness function combining MSE and CCE
def evaluate_fitness(individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)

    # Calculate MSE
    mse = calculate_mse(predictions, y_train)

    # Calculate CCE
    cce = calculate_cce(predictions, y_train)

    # Combine MSE and CCE with weights
    fitness = mse_weight * mse + cce_weight * cce
    return fitness

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

# Set the best weights found
best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_mse = calculate_mse(test_predictions, y_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.losses import categorical_crossentropy

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the IMDB dataset
max_features = 20000
maxlen = 200  # Cut texts after this number of words (among top max_features most common words)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

# One-hot encoding the labels
encoder = OneHotEncoder(sparse=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# Create a neural network architecture
model = Sequential([
    Embedding(max_features, 128, input_length=maxlen),
    LSTM(128, return_sequences=True),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # Output layer for 2 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05
mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to calculate MSE for fitness
def calculate_mse(predictions, true_values):
    return np.mean(np.square(predictions - true_values))

# Function to calculate CCE for fitness
def calculate_cce(predictions, true_values):
    return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# Hybrid fitness function combining MSE and CCE
def evaluate_fitness(individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)

    # Calculate MSE
    mse = calculate_mse(predictions, y_train)

    # Calculate CCE
    cce = calculate_cce(predictions, y_train)

    # Combine MSE and CCE with weights
    fitness = mse_weight * mse + cce_weight * cce
    return fitness

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

# Set the best weights found
best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_mse = calculate_mse(test_predictions, y_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Early stopping at generation 15
1/1 [==============================] - 0s 24ms/step
Test set MSE: 0.1942
Test set accuracy: 0.70


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Embedding, Flatten
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.datasets import imdb
# from sklearn.preprocessing import OneHotEncoder
# from tensorflow.keras.losses import categorical_crossentropy

# # Set random seed for reproducibility
# np.random.seed(42)

# # Load and preprocess the IMDB dataset
# max_features = 10000  # Top 10,000 words
# maxlen = 200  # Maximum sequence length

# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
# X_train = pad_sequences(X_train, maxlen=maxlen)
# X_test = pad_sequences(X_test, maxlen=maxlen)

# # One-hot encode the labels
# encoder = OneHotEncoder(sparse=False)
# y_train = encoder.fit_transform(y_train.reshape(-1, 1))
# y_test = encoder.transform(y_test.reshape(-1, 1))

# # Create an ANN model
# def create_ann_model():
#     model = Sequential([
#         Embedding(max_features, 128, input_length=maxlen),
#         Flatten(),  # Flatten the embeddings for fully connected layers
#         Dense(128, activation='relu'),
#         Dense(64, activation='relu'),
#         Dense(2, activation='softmax')  # Binary classification
#     ])
#     return model

# # Genetic algorithm parameters
# population_size = 20
# generations = 20
# mutation_rate = 0.05
# mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
# cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# # Function to create random weights for the model
# def create_individual(model):
#     return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# # Function to calculate MSE for fitness
# def calculate_mse(predictions, true_values):
#     return np.mean(np.square(predictions - true_values))

# # Function to calculate CCE for fitness
# def calculate_cce(predictions, true_values):
#     return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# # Hybrid fitness function combining MSE and CCE
# def evaluate_fitness(model, individual):
#     model.set_weights(individual)
#     predictions = model.predict(X_train, verbose=0)

#     # Calculate MSE and CCE
#     mse = calculate_mse(predictions, y_train)
#     cce = calculate_cce(predictions, y_train)

#     # Combine MSE and CCE with weights
#     fitness = mse_weight * mse + cce_weight * cce
#     return fitness

# # Function to perform crossover between two parents
# def crossover(parent1, parent2):
#     child = []
#     for p1, p2 in zip(parent1, parent2):
#         mask = np.random.rand(*p1.shape) > 0.5
#         child.append(np.where(mask, p1, p2))
#     return child

# # Function to perform mutation on an individual
# def mutate(individual, rate):
#     for i in range(len(individual)):
#         if np.random.rand() < rate:
#             mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
#             individual[i] += mutation
#     return individual

# # Initialize the population
# model = create_ann_model()
# population = [create_individual(model) for _ in range(population_size)]

# # Genetic algorithm loop
# for generation in range(generations):
#     fitness_scores = [evaluate_fitness(model, ind) for ind in population]

#     # Elitism: Carry forward the best individual
#     best_individual = population[np.argmin(fitness_scores)]
#     new_population = [best_individual]  # Start new population with the best individual

#     selected_indices = np.argsort(fitness_scores)[:population_size // 2]
#     selected_population = [population[i] for i in selected_indices]

#     while len(new_population) < population_size:
#         parents = np.random.choice(range(len(selected_population)), 2, replace=False)
#         child = crossover(selected_population[parents[0]], selected_population[parents[1]])
#         child = mutate(child, mutation_rate)
#         new_population.append(child)

#     population = new_population

# # Set the best weights found
# best_weights = population[np.argmin([evaluate_fitness(model, ind) for ind in population])]
# model.set_weights(best_weights)

# # Evaluate the model on the test set
# test_predictions = model.predict(X_test)
# test_mse = calculate_mse(test_predictions, y_test)
# test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

# print(f"Test set MSE: {test_mse:.4f}")
# print(f"Test set accuracy: {test_accuracy:.2f}")


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: You called `set_weights(weights)` on layer 'sequential' with a weight list of length 0, but the layer was expecting 7 weights.

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from datasets import load_dataset

# Load and preprocess the IMDB dataset from Hugging Face
dataset = load_dataset('imdb')

# Preprocessing and feature extraction
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']

# Split train into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = CountVectorizer(stop_words='english', max_features=5000)
X_train = vectorizer.fit_transform(train_texts).toarray()
X_val = vectorizer.transform(val_texts).toarray()
X_test = vectorizer.transform(test_texts).toarray()

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
y_val_tensor = torch.tensor(val_labels, dtype=torch.long)
y_test_tensor = torch.tensor(test_labels, dtype=torch.long)

class IMDBDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            'inputs': self.texts[idx],
            'labels': self.labels[idx]
        }

# Prepare datasets and dataloaders
train_dataset = IMDBDataset(X_train_tensor, y_train_tensor)
val_dataset = IMDBDataset(X_val_tensor, y_val_tensor)
test_dataset = IMDBDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the ANN model with L1 pruning
class ANNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_units, output_dim):
        super(ANNModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_units)
        self.fc2 = torch.nn.Linear(hidden_units, hidden_units)
        self.fc3 = torch.nn.Linear(hidden_units, output_dim)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def create_model(hidden_units, learning_rate):
    input_dim = X_train.shape[1]  # Number of features
    output_dim = 2  # Binary classification
    model = ANNModel(input_dim, int(hidden_units), output_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    return model, optimizer

# Objective Function: Minimize Validation Loss with L1 Regularization
def objective_function(hyperparams):
    hidden_units = hyperparams[0]
    learning_rate = hyperparams[1]

    model, optimizer = create_model(hidden_units, learning_rate)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)

    # Train the model for one epoch
    model.train()
    for batch in train_loader:
        inputs = batch['inputs']
        labels = batch['labels']
        optimizer.zero_grad()
        outputs = model(inputs)

        # Cross-entropy loss
        loss = torch.nn.CrossEntropyLoss()(outputs, labels)

        # L1 Regularization
        l1_lambda = 0.001  # Adjust as needed
        l1_norm = sum(torch.norm(param, 1) for param in model.parameters())
        loss += l1_lambda * l1_norm

        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    model.eval()
    preds = []
    labels_list = []
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['inputs']
            labels = batch['labels']
            outputs = model(inputs)
            preds += torch.argmax(outputs, dim=1).tolist()
            labels_list += labels.tolist()

    accuracy = accuracy_score(labels_list, preds)
    return 1 - accuracy  # Minimize this value

# Bat Algorithm Implementation
class BatAlgorithm:
    def __init__(self, num_bats, max_iter, bounds, alpha=0.9, gamma=0.9):
        self.num_bats = num_bats
        self.max_iter = max_iter
        self.bounds = bounds
        self.alpha = alpha
        self.gamma = gamma

        # Initialize bats
        self.bats = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_bats, bounds.shape[0]))
        self.velocities = np.zeros((num_bats, bounds.shape[0]))
        self.fitness = np.zeros(num_bats)
        self.frequencies = np.zeros(num_bats)

        # Best bat initialization
        self.best_bat = self.bats[0]
        self.best_fitness = float("inf")

    def optimize(self, objective_function):
        for iter in range(self.max_iter):
            for i in range(self.num_bats):
                # Frequency update
                self.frequencies[i] = np.random.uniform(0, 1)
                # Velocity update
                self.velocities[i] = self.velocities[i] + (self.bats[i] - self.best_bat) * self.frequencies[i]
                # Position update
                candidate_bat = self.bats[i] + self.velocities[i]
                candidate_bat = np.clip(candidate_bat, self.bounds[:, 0], self.bounds[:, 1])

                # Evaluate candidate
                fitness = objective_function(candidate_bat)

                # Update if new solution is better
                if fitness < self.fitness[i]:
                    self.bats[i] = candidate_bat
                    self.fitness[i] = fitness

                # Update the best bat
                if self.fitness[i] < self.best_fitness:
                    self.best_bat = self.bats[i]
                    self.best_fitness = self.fitness[i]

            print(f"Iteration {iter+1}/{self.max_iter}, Best Fitness: {self.best_fitness}")

        return self.best_bat

# Hyperparameter bounds: hidden units (4 to 64), learning rate (1e-6 to 1e-2)
bounds = np.array([[4, 64], [1e-6, 1e-2]])

# Initialize and run Bat Algorithm
bat_algorithm = BatAlgorithm(num_bats=10, max_iter=10, bounds=bounds, alpha=0.9, gamma=0.9)
best_hyperparams = bat_algorithm.optimize(objective_function)
print("Best hyperparameters found:", best_hyperparams)

# Use the best hyperparameters to train the final model
best_hidden_units, best_lr = best_hyperparams
final_model, final_optimizer = create_model(best_hidden_units, best_lr)
final_train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train and Evaluate Final Model with Best Hyperparameters
for epoch in range(3):  # You can increase the number of epochs if needed
    final_model.train()
    for batch in final_train_loader:
        inputs = batch['inputs']
        labels = batch['labels']
        final_optimizer.zero_grad()
        outputs = final_model(inputs)

        # Cross-entropy loss
        loss = torch.nn.CrossEntropyLoss()(outputs, labels)

        # L1 Regularization
        l1_norm = sum(torch.norm(param, 1) for param in final_model.parameters())
        loss += l1_lambda * l1_norm

        loss.backward()
        final_optimizer.step()
    print(f"Epoch {epoch+1} completed.")

# Evaluate final model
final_model.eval()
test_loader = DataLoader(test_dataset, batch_size=32)
preds = []
labels_list = []
with torch.no_grad():
    for batch in test_loader:
        inputs = batch['inputs']
        labels = batch['labels']
        outputs = final_model(inputs)
        preds += torch.argmax(outputs, dim=1).tolist()
        labels_list += labels.tolist()

accuracy = accuracy_score(labels_list, preds)
print(f"Final model accuracy: {accuracy:.2f}")


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.losses import categorical_crossentropy

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a larger neural network architecture
model = Sequential([
    Dense(64, input_dim=4, activation='relu'),   # Increased units
    Dense(128, activation='relu'),                # Increased units
    Dense(64, activation='relu'),                 # Increased units
    Dense(32, activation='relu'),                 # New layer added
    Dense(3, activation='softmax')                # Output layer for 3 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05
mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to calculate MSE for fitness
def calculate_mse(predictions, true_values):
    return np.mean(np.square(predictions - true_values))

# Function to calculate CCE for fitness
def calculate_cce(predictions, true_values):
    return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# Hybrid fitness function combining MSE and CCE
def evaluate_fitness(individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)

    # Calculate MSE
    mse = calculate_mse(predictions, y_train)

    # Calculate CCE
    cce = calculate_cce(predictions, y_train)

    # Combine MSE and CCE with weights
    fitness = mse_weight * mse + cce_weight * cce
    return fitness

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

# Set the best weights found
best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_mse = calculate_mse(test_predictions, y_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.losses import categorical_crossentropy

# Set random seed for reproducibility
np.random.seed(42)

# Load and preprocess the IMDB dataset
max_features = 20000
maxlen = 200  # Cut texts after this number of words (among top max_features most common words)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

# One-hot encoding the labels
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

# Create a neural network architecture
model = Sequential([
    Embedding(max_features, 128, input_length=maxlen),
    LSTM(128, return_sequences=True),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # Output layer for 2 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define genetic algorithm parameters
population_size = 20
generations = 20
mutation_rate = 0.05
mse_weight = 0.4  # Weight for MSE in the hybrid fitness function
cce_weight = 0.6  # Weight for categorical crossentropy in the hybrid fitness function

# Function to create random weights for the model
def create_individual():
    return [np.random.uniform(-1, 1, w.shape) for w in model.get_weights()]

# Function to calculate MSE for fitness
def calculate_mse(predictions, true_values):
    return np.mean(np.square(predictions - true_values))

# Function to calculate CCE for fitness
def calculate_cce(predictions, true_values):
    return np.mean(categorical_crossentropy(true_values, predictions).numpy())

# Hybrid fitness function combining MSE and CCE
def evaluate_fitness(individual):
    model.set_weights(individual)
    predictions = model.predict(X_train, verbose=0)

    # Calculate MSE
    mse = calculate_mse(predictions, y_train)

    # Calculate CCE
    cce = calculate_cce(predictions, y_train)

    # Combine MSE and CCE with weights
    fitness = mse_weight * mse + cce_weight * cce
    return fitness

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = []
    for p1, p2 in zip(parent1, parent2):
        mask = np.random.rand(*p1.shape) > 0.5
        child.append(np.where(mask, p1, p2))
    return child

# Function to perform mutation on an individual
def mutate(individual, rate):
    for i in range(len(individual)):
        if np.random.rand() < rate:
            mutation = np.random.uniform(-0.5, 0.5, individual[i].shape)
            individual[i] += mutation
    return individual

# Initialize the population
population = [create_individual() for _ in range(population_size)]

# Early stopping parameters
early_stopping_rounds = 5
no_improvement_count = 0
best_fitness = np.inf

# Genetic algorithm loop with early stopping
for generation in range(generations):
    fitness_scores = [evaluate_fitness(ind) for ind in population]

    # Check for improvement
    current_best_fitness = min(fitness_scores)
    if current_best_fitness < best_fitness:
        best_fitness = current_best_fitness
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_rounds:
        print(f"Early stopping at generation {generation}")
        break

    # Elitism: Carry forward the best individual
    best_individual = population[np.argmin(fitness_scores)]
    new_population = [best_individual]  # Start new population with the best individual

    selected_indices = np.argsort(fitness_scores)[:population_size // 2]
    selected_population = [population[i] for i in selected_indices]

    while len(new_population) < population_size:
        parents = np.random.choice(range(len(selected_population)), 2, replace=False)
        child = crossover(selected_population[parents[0]], selected_population[parents[1]])
        child = mutate(child, mutation_rate)
        new_population.append(child)

    population = new_population

# Set the best weights found
best_weights = population[np.argmin([evaluate_fitness(ind) for ind in population])]
model.set_weights(best_weights)

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
test_mse = calculate_mse(test_predictions, y_test)
test_accuracy = np.mean(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1))

print(f"Test set MSE: {test_mse:.4f}")
print(f"Test set accuracy: {test_accuracy:.2f}")



/Users/chethanarkini/miniforge3/envs/rakuten/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Early stopping at generation 15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Test set MSE: 0.1942
Test set accuracy: 0.70


/Users/chethanarkini/miniforge3/envs/rakuten/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: You called `set_weights(weights)` on layer 'sequential_3' with a weight list of length 0, but the layer was expecting 8 weights.